## Tutoriel pytorch 
### Torchtext overview

Torchtext is a Python library that provides a suite of tools for natural language processing (NLP) tasks such as text preprocessing, tokenization, and vocabulary management. It is specifically designed for working with text data and provides several functions for cleaning and preparing text data, including removing punctuation, extra whitespaces, and other special characters, splitting text into words or tokens, and converting text to lowercase.

torchtext is often used in conjunction with PyTorch, a popular deep learning framework, to build end-to-end NLP pipelines. It can be particularly useful for preparing text data for tasks such as text classification, sentiment analysis, and machine translation.

The for acces to torchtext you need to dowload the library using the next command below.

In [24]:
! pip install pytorch
! pip install torchtext
! pip install transformers
! pip install SentencePiece

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-wf7iwzkg/pytorch_faf10091f97c497cba8e9cdbd4e51aaa/setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
  Running setup.py install for pytorch ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for pytorch did not run successfully.
  │ exit co

### Tokenizer
Tokenizer is an approach used in natural language processing (nlp) to split a text in to token. 
Tokenization is the process of breaking down a text into individual words, phrases, symbols, or other meaningful elements, which are referred to as "tokens". These tokens can then be used for a variety of NLP tasks, such as language modeling, text classification, and sentiment analysis.
For load it you need [torchtext.data.utils]

#### get_tokenizer 
get_tokenizer from torchtext.data.utils is a method for tokenize text 
tokenizer = get_tokenizer(tokenizer method ,  language )

In [2]:
from torchtext.data.utils import get_tokenizer
def get_english_tokenizer():
    """
    Documentation:
    https://pytorch.org/text/stable/_modules/torchtext/data/utils.html#get_tokenizer
    """
    tokenizer = get_tokenizer("basic_english", language="en")

    return tokenizer

## Vocab
vocab is a set of word with their idx : 
words ["je" , "vais" , "sortir"]
vocab(words) ----> {"je" : 0 ; "vais" :  1, "sortir" : 2}

Then, you need to tokenize first the text befor using vocab. 

For use vocab in pytorch, you use the following code snippet.

It take two parameters such as: data wich can be a list of words (["je suis parti" , "Il est rentré"]) and the tokenizer function.
add special token for the unknow word with parameter (specials = ["<unk>"] )


In [ ]:
from torchtext.vocab import build_vocab_from_iterator
def build_vocab(data_iter, tokenizer):
    """Builds vocabulary from iterator"""
    
    vocab = build_vocab_from_iterator(
        map(tokenizer, data_iter),
        specials=["<unk>"],
        min_freq=1,
    )
    vocab.set_default_index(vocab["<unk>"])
    return vocab


### CONTEXT IN NATURAL LANGUAGE PROICESSING
Suppose you are reading a book and you come across a line of text that you don't understand. What can you do to understand it? Firstly, you should reread the entire sentence and look at the words surrounding the unfamiliar word. Once you have a clearer understanding of the context, you can then work out the meaning of the unknown word. This method of understanding an unknown word by using the context of the text is known as contextualization.

Contextualization in NLP is so important for the model to understand well the whole sentence and make the relation between all words inside the sentence.
The parameter than we use such a way to control the number of the word that we are chose before and after the unknow word (current word) is called N_CONTEXT_WORDS

For by the know the entire lenght of the word tou should do this kind of operation N_CONTEXT_WORDS * 2 + 1: This max sequence length
Batch means a bag of sentences. 

In [19]:
CBOW_N_WORDS = 2
MAX_SEQUENCE_LENGTH = CBOW_N_WORDS * 2 +1

## CBOW (CONTEXTUAL BAG OF WORD)

In [20]:

def collate_cbow(batch ):
    """
    Collate_fn for CBOW model to be used with Dataloader.
    `batch` is expected to be list of text paragrahs.
    
    Context is represented as N=CBOW_N_WORDS past words 
    and N=CBOW_N_WORDS future words.
    
    Long paragraphs will be truncated to contain
    no more that MAX_SEQUENCE_LENGTH tokens.
    
    Each element in `batch_input` is N=CBOW_N_WORDS*2 context words.
    Each element in `batch_output` is a middle word.
    """
    data_iter = batch
    tokenizer = get_english_tokenizer()

   
    vocab = build_vocab(data_iter, tokenizer)
        
    text_pipeline = lambda x: vocab(tokenizer(x))

    batch_input, batch_output = [], []
    text_pipeline = lambda x: vocab(tokenizer(x))
    for text in batch:
        text_tokens_ids =text_pipeline (text)

        if len(text_tokens_ids) < CBOW_N_WORDS * 2 + 1:
            continue

        if MAX_SEQUENCE_LENGTH:
            text_tokens_ids = text_tokens_ids[:MAX_SEQUENCE_LENGTH]
            print(text_tokens_ids)

        for idx in range(len(text_tokens_ids) - CBOW_N_WORDS * 2):
            token_id_sequence = text_tokens_ids[idx : (idx + CBOW_N_WORDS * 2 + 1)]
            output = token_id_sequence.pop(CBOW_N_WORDS)
            input_ = token_id_sequence
            batch_input.append(input_)
            batch_output.append(output)
    print(batch_input)
    print(batch_output)
    batch_input = torch.tensor(batch_input, dtype=torch.long)
    batch_output = torch.tensor(batch_output, dtype=torch.long)
    return batch_input, batch_output

collate_cbow(batch)

[5, 11, 1, 8, 13]
[4, 2, 10, 9, 12]
[[5, 11, 8, 13], [4, 2, 9, 12]]
[1, 10]


(tensor([[ 5, 11,  8, 13],
         [ 4,  2,  9, 12]]),
 tensor([ 1, 10]))